## Discretisation techniques

Here we consider the discretization of $H = - \frac 1 2 \Delta + V$ with $V(x) = \cos(x)$ on $[0, 2\pi]$ with periodic boundary conditions. (note that this is different from e.g. looking for the spectrum of $H$ on $\mathbb R$).

## Finite differences
We approximate functions on $[0, 2\pi]$ by their values at grid points $x_k = 2\pi \frac{k}{N}$, $k=1, \dots, N$. The boundary conditions are imposed by $\psi(x_0) = \psi(x_N), \psi(x_{N+1}) = \psi(x_0)$. We then have
$$(H\psi)(x_k) \approx \frac 1 2 \frac{-\psi_{k-1} + 2 \psi_k - \psi_{k+1}}{2 \Delta x^2} + V(x_k) \psi(x_k)$$ with $\Delta x = 2 \pi \frac 1 N$.

This can be put in matrix form in the following way:

In [ ]:
# finite differences Hamiltonian -1/2 Delta + V on [0, 2pi] with periodic bc. Pass it a function V.
function build_finite_differences_matrix(N, V)
    Δx = 2π/N
    # build the finite difference matrix. We could also build it iteratively with a for k=1:N ... end loop
    T = 1/(2*Δx^2) * Tridiagonal(-ones(N-1), 2ones(N), -ones(N-1))
    # The type Tridiagonal is efficient, but we want to add elements
    # not on the three diagonals, so convert to Array
    T = Array(T)
    T[1,N] = T[N,1] = -1/(2*Δx^2)  # periodic bc
    x_coords = [2π*k/N for k=1:N]
    Vvals = V.(x_coords)  # calls V on each element of x_coords
    # the types Tridiagonal and Diagonal are efficient but not necessarily easy
    # to manipulate interactively, so we convert to plain old dense arrays
    Array(T + Diagonal(Vvals)) 
end

**Exercice**: show that the finite difference approximation is an approximation of the second derivative. Obtain an estimate of the first eigenvalue of $H$.

## Plane waves method

In this method, we expand states on the basis $e_G(x) = \frac {1}{\sqrt{2\pi}} e^{iGx}$, for $G=-N,\dots,N$.

**Exercice**: show that $\langle e_G, e_{G'}\rangle = \delta_{G,G'}$, and $$\langle e_G, H e_{G'}\rangle = \frac 1 2 |G|^2 \delta_{G,G'}+ \delta_{G, G'+1} + \delta_{G, G'-1}.$$ What happens for a more general $V(x)$?

**Exercice**: code this, and check that this corresponds to the finite difference case. Compare accuracies.

## Using DFTK
### Setup

In [ ]:
using DFTK
using LinearAlgebra
using Plots

# We deal with a 1D system. This is done in DFTK by having a 3D lattice with two lattice vectors set to zero.
a = 2π
lattice = a .* [[1 0 0.]; [0 0 0]; [0 0 0]];

# define Hamiltonian = kinetic + potential
terms = [Kinetic(),
         ExternalFromReal(r -> cos(r[1])),  # r is a vector of size 3
]
model = Model(lattice; n_electrons=1, terms, spin_polarization=:spinless);  # one spinless electron

# this defines the number of plane waves (N above) by the relationship 1/2 |G|^2 <= N
Ecut = 500
basis = PlaneWaveBasis(model; Ecut, kgrid=(1, 1, 1))
# get the ground state. We set the diagtol (the tolerance of the eigensolver) to a small value
# to better separate the two steps (SCF outer loop, and diagonalization inner loop)
scfres = self_consistent_field(basis, tol=1e-4, determine_diagtol=DFTK.ScfDiagtol(diagtol_max=1e-8))
scfres.energies

On this simple linear (non-interacting) model, the SCF converges in one step. The ground state energy of is simply the lowest eigenvalue; it should match the smallest eigenvalue of $H$ computed above.

### Comparison

We can also get the first eigenvector (in the plane wave basis) and plot it

In [ ]:
ψ_fourier = scfres.ψ[1][:, 1];    # first k-point, all G components, first eigenvector
# Transform the wave function to real space
ψ = G_to_r(basis, basis.kpoints[1], ψ_fourier)[:, 1, 1]
# eigenvectors are only defined up to a phase. We fix it by imposing that psi(0) is real
ψ /= (ψ[1] / abs(ψ[1]))
plot(real(ψ))

Again, it should match with the result above.

**Exercice**: look at the Fourier coefficients of $\psi$ ($\psi$_fourier) and compare with the result above.

## The DFTK Hamiltonian
We can ask DFTK for the Hamiltonian

In [ ]:
E, ham = energy_hamiltonian(basis, scfres.ψ, scfres.occupation; ρ=scfres.ρ)
H = ham.blocks[1]
typeof(H)

This is an opaque data structure, which encodes the Hamiltonian. What can we do with it?

In [ ]:
methodswith(typeof(H), supertypes=true)

This defines a number of methods. For instance, it can be used as a linear operator:

In [ ]:
H * randn(ComplexF64, length(G_vectors(basis, basis.kpoints[1])))

We can also get its full matrix representation:

In [ ]:
Array(H)

**Exercice**: compare this matrix with the one you obtained previously, get its eigenvectors and eigenvalues. Try to guess the ordering of G vectors in DFTK.

**Exercice**: increase the size of the problem, and compare the time spent by DFTK's internal diagonalization algorithms to a full diagonalization of Array(H).